# Dimensionality reduction with T-SNE


In [8]:
import os
import sys
import numpy as np
import sklearn
import keras as K
from keras.applications.nasnet import NASNetLarge
from keras.applications.resnet50 import ResNet50
import tensorflow as tf
import t_sne_bhcuda.t_sne_bhcuda.bhtsne_cuda as tsne_bhcuda
from utils import plot_tsne, get_gpu_name, get_cuda_version, get_cudnn_version

print("System version: {}".format(sys.version))
print("Sklearn version: {}".format(sklearn.__version__))
print("Numpy version: {}".format(np.__version__))
print("Keras version: {}".format(K.__version__))
print("Keras backend: {}".format(K.backend.backend()))
print("Keras image data format: {}".format(K.backend.image_data_format()))
print("Tensorflow version: {}".format(tf.__version__))
print("GPU: {}".format(get_gpu_name()))
print("CUDA version: {}".format(get_cuda_version()))
print("CuDNN version: {}".format(get_cudnn_version()))

%load_ext autoreload
%autoreload 2

System version: 3.5.5 |Anaconda custom (64-bit)| (default, May 13 2018, 21:12:35) 
[GCC 7.2.0]
Sklearn version: 0.19.1
Numpy version: 1.14.5
Keras version: 2.2.2
Keras backend: tensorflow
Keras image data format: channels_last
Tensorflow version: 1.10.1
GPU: ['Tesla K80']
CUDA version: CUDA Version 9.2.148
CuDNN version: 7.2.1
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Dataset

[Dogs vs Cats](https://www.kaggle.com/c/dogs-vs-cats/data) dataset, which contains 2 classes and 25000 images.


In [11]:
import kaggle

OSError: Could not find kaggle.json. Make sure it's located in /home/hoaphumanoid/.kaggle. Or use the environment method.